<a href="https://colab.research.google.com/github/DanielBrisch/Heartdiseases/blob/03/3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
!pip install ucimlrepo
from ucimlrepo import fetch_ucirepo
doenca_cardiaca = fetch_ucirepo(id=45)

In [3]:
X = doenca_cardiaca.data.features
X = X[['age', 'chol', 'cp']]

In [4]:
target = doenca_cardiaca.data.targets
target = (target > 0) * 1

In [5]:
df_cp = pd.get_dummies(X['cp'], prefix='cp')
X = X.drop('cp', axis=1).join(df_cp)

In [6]:
y = tf.constant(target, dtype=tf.float32)

In [7]:
X.head()

,age,chol,cp_1,cp_2,cp_3,cp_4
0,63,233,1,0,0,0
1,67,286,0,0,0,1
2,67,229,0,0,0,1
3,37,250,0,0,1,0
4,41,204,0,1,0,0


In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
scaler = StandardScaler()

In [10]:
X[['age', 'chol']] = scaler.fit_transform(X[['age', 'chol']])

print('primeiros valores normalizados de age:', X['age'].head())
print('primeiros valores normalizados de chol: ', X['chol'].head())

primeiros valores normalizados de age: 0    0.948726
1    1.392002
2    1.392002
3   -1.932564
4   -1.489288
Name: age, dtype: float64
primeiros valores normalizados de chol:  0   -0.264900
1    0.760415
2   -0.342283
3    0.063974
4   -0.825922
Name: chol, dtype: float64


In [11]:
X.shape[1]

6

In [12]:
tf.random.set_seed(31)

In [13]:
quantidade_features = X.shape[1]
peso = tf.Variable(tf.random.normal([quantidade_features, 1]), name='peso')
vies = tf.Variable(tf.random.normal([1]), name='vies')

In [14]:
def neuronio(x):
  z = tf.add(tf.matmul(x, peso), vies)
  return z

In [15]:
vies.numpy(), peso.numpy()

(array([0.15100643], dtype=float32),
 array([[-0.3594139 ],
        [ 0.18532915],
        [ 2.3871737 ],
        [ 0.9996345 ],
        [ 0.09981808],
        [-0.06186233]], dtype=float32))

In [16]:
X.iloc[0].values

array([ 0.94872647, -0.2649003 ,  1.        ,  0.        ,  0.        ,
        0.        ])

In [17]:
neuronio(tf.constant([[0.94872647, -0.2649003, 1.0, 0.0, 0.0, 0.0]]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[2.1481009]], dtype=float32)>

In [18]:
pb = tf.sigmoid(neuronio(tf.constant([[0.94872647, -0.2649003, 1.0, 0.0, 0.0, 0.0]])))
pb

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.8954912]], dtype=float32)>

In [19]:
entrada = tf.constant(X, dtype=tf.float32)
pb = tf.sigmoid(neuronio(entrada))
pb[:10].numpy()

array([[0.8954912 ],
       [0.43284974],
       [0.38352907],
       [0.7225751 ],
       [0.82242215],
       [0.7407992 ],
       [0.46614692],
       [0.5919096 ],
       [0.44382718],
       [0.4974511 ]], dtype=float32)

In [20]:
calc_perda = tf.keras.losses.BinaryCrossentropy()

In [21]:
perda = calc_perda(y_true = y, y_pred = pb)
perda.numpy()

0.9295621

In [22]:
np.mean(y.numpy() == ((pb.numpy() > 0.5) * 1))

0.3696369636963696

In [23]:
tf.random.set_seed(31)

quantidade_features = X.shape[1]

pesos = tf.Variable(tf.random.normal([quantidade_features,1]), name='pesos')
vies = tf.Variable(tf.random.normal([1]), name='vies')

def neuronio(x):
  z = tf.add(tf.matmul(x, pesos), vies)
  return z

calculadora_perda = tf.keras.losses.BinaryCrossentropy()


In [24]:
entrada = tf.constant(X, dtype=tf.float32)

In [25]:
entrada[:3], y[:3]

(<tf.Tensor: shape=(3, 6), dtype=float32, numpy=
 array([[ 0.9487265, -0.2649003,  1.       ,  0.       ,  0.       ,
          0.       ],
        [ 1.3920019,  0.7604152,  0.       ,  0.       ,  0.       ,
          1.       ],
        [ 1.3920019, -0.3422826,  0.       ,  0.       ,  0.       ,
          1.       ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 1), dtype=float32, numpy=
 array([[0.],
        [1.],
        [1.]], dtype=float32)>)

In [26]:
learning_rate = 0.01

otimizador = tf.optimizers.SGD(learning_rate=learning_rate)

In [27]:
QUANTIDADE_EPOCAS = 1000

In [28]:
for epoca in range(QUANTIDADE_EPOCAS):
  with tf.GradientTape() as tape:
    pb = tf.sigmoid(neuronio(entrada))
    custo = calc_perda(y, pb)


  gradientes = tape.gradient(custo, [pesos, vies])
  otimizador.apply_gradients(zip(gradientes, [pesos, vies]))

  if (epoca + 1) % 100 == 0:
    print(f'Época: {epoca+1}, Custo: {custo.numpy()}')


Época: 100, Custo: 0.8540225625038147
Época: 200, Custo: 0.7985237836837769
Época: 300, Custo: 0.7575094699859619
Época: 400, Custo: 0.7262797951698303
Época: 500, Custo: 0.7017260193824768
Época: 600, Custo: 0.6818602681159973
Época: 700, Custo: 0.6654065847396851
Época: 800, Custo: 0.6515259742736816
Época: 900, Custo: 0.6396480798721313
Época: 1000, Custo: 0.6293708086013794


In [30]:
np.mean(y.numpy() == ((pb.numpy() > 0.5) * 1))

0.7161716171617162